In [41]:
from ete3 import Tree as eTree
from dendropy import Tree
import pandas as pd
'''
For the tree analysis, I am thinking that we could 
measure precision and recall as well! The way to do 
this is to collapse all the branches which are below a 
certain support value threshold (say, 0.9). 
Precision is given by [#edges in inferred trees that are also in reference tree / #edges in inferred trees] and 
Recall is given by [#edges in inferred trees that are also in reference tree / #edges in reference trees]'''

bootstrap_threshold = 0.9

In [83]:
r2t = '(SORBI:0.0144438823,(((((((SELML:0.2094553519,((CHLVA:0.3424741526,KLEFL:0.2117930540)100:0.0964109982,PHYPA:0.1759691783)100:0.0432292079)100:0.1510909698,AMBTC:0.1208203262)100:0.0414459882,((((POPTR:0.0727693188,MANES:0.0587676581)100:0.0260358918,(GOSHI:0.0462487758,THECC:0.0330931724)100:0.0479364023)75:0.0220311497,VITVI:0.0733045624)73:0.0202357100,(ARATH:0.0759490217,(SOLLC:0.0104105805,SOLTU:0.0088545369)100:0.0846056583)73:0.0296613578)100:0.0478788151)100:0.0524628131,(MUSAM:0.0000012286,MUSAC:0.0000012286)100:0.1010642542)100:0.1198415813,((((ORYNI:0.0017949225,ORYRU:0.0020199265)100:0.0024959124,ORYLO:0.0133485493)100:0.0058761396,ORYPU:0.0090841121)100:0.0388290701,((((TRIUA:0.0132910953,AEGTA:0.0114460951)88:0.0021234647,WHEAT:0.0020768605)100:0.0072601838,HORVD:0.0107965823)100:0.0301828014,BRADI:0.0299010011)100:0.0338024384)100:0.0101921826)100:0.0161997732,ERATE:0.0441966807)100:0.0148427621,SETIT:0.0236409275)100:0.0167360497,MAIZE:0.0231870249);'
ref = '(VITVI:0.08986284816204641,(((((((ORYLO:0.017559398833091405,(ORYNI:0.0030331844454580897,ORYRU:0.003224499997376138)100:0.003746597082533012)100:0.008765551141179311,ORYPU:0.013223441191895991)100:0.05107033503836343,((((TRIUA:0.02003035653962613,AEGTA:0.0161576391580634)100:0.0030393170797259946,WHEAT:0.002486556823229595)100:0.009630324600537593,HORVD:0.013455061321712335)100:0.03944508303185502,BRADI:0.03639955100062263)100:0.04232978775837709)100:0.012741988243246204,(ERATE:0.05554088136449878,((SORBI:0.017087823953061066,MAIZE:0.028629215272852317)100:0.02112795401841375,SETIT:0.028877298850367734)100:0.01851860045465869)100:0.02028070384120017)100:0.1452209852570855,(MUSAM:8.792510036786545e-07,MUSAC:8.792510036786545e-07)100:0.12364255015100582)100:0.06254865077415761,(AMBTC:0.14957762510445285,(SELML:0.2482955444045423,((KLEFL:0.2744631655567387,CHLVA:0.5181232204685107)100:0.1307629912808552,PHYPA:0.2276359754731672)100:0.04991593866501401)100:0.17428205734239413)100:0.046412722787747364)100:0.057248346683988954,(SOLLC:0.012438031840102056,SOLTU:0.01090075741120105)100:0.13786079517011832)100:0.023254722771489714,((MANES:0.07243268739339274,POPTR:0.08935402473159693)100:0.03237413514165099,(ARATH:0.18929054785106217,(THECC:0.04106199938900029,GOSHI:0.05635376409464054)100:0.05145650578345483)100:0.01681607416947823)100:0.026329168732743605):0.0;'




In [84]:
r2t_tree = Tree.get(data=r2t, schema='newick')# .get(data="((A,B),(C,D));", schema="newick")
ref_tree = Tree.get(data=ref, schema='newick')

r2t_tree = eTree(r2t)# .get(data="((A,B),(C,D));", schema="newick")
ref_tree = eTree(ref)

In [92]:
r2t_matrix = {i:[] for i in sorted(r2t_tree.get_leaf_names())}
ref_matrix = {i:[] for i in sorted(ref_tree.get_leaf_names())}

In [93]:
for node in r2t_tree.traverse("postorder"):
  # Do some analysis on node
    if len(node.get_leaf_names()) > 1 and len(node.get_leaf_names()) < len(r2t_tree.get_leaf_names()):
#         print(node.support)
#         print(node.get_leaf_names())
        if node.support > 90:
            for k,v in r2t_matrix.items():
                if k in node.get_leaf_names():
                    v.append('1')
                else:
                    v.append('0')
r2t_df = pd.DataFrame(r2t_matrix)

In [94]:
for node in ref_tree.traverse("postorder"):
  # Do some analysis on node
    if len(node.get_leaf_names()) > 1 and len(node.get_leaf_names()) < len(ref_tree.get_leaf_names()):
#         print(node.support)
#         print(node.get_leaf_names())
        if node.support > 90:
            for k,v in ref_matrix.items():
                if k in node.get_leaf_names():
                    v.append('1')
                else:
                    v.append('0')
ref_df = pd.DataFrame(ref_matrix)

In [95]:
ref_str = [''.join(list(r)) for i,r in ref_df.iterrows()]
r2t_str = [''.join(list(r)) for i,r in r2t_df.iterrows()]

In [102]:
edges_in_r2t = 0
for r2t_i in r2t_str:
    if r2t_i in ref_str:
        edges_in_r2t = edges_in_r2t + 1
precision = edges_in_r2t / len(r2t_str)

In [103]:
edges_in_ref = 0
for ref_i in ref_str:
    if ref_i in r2t_str:
        edges_in_ref = edges_in_ref + 1
recall = edges_in_ref / len(ref_str)